In [ ]:
import { GoogleGenerativeAI } from "@google/generative-ai";
import "std/dotenv/load.ts";

// 1. AI가 실행할 실제 함수 정의
const functions: Record<string, (args: any) => any> = {
    get_current_weather: ({ location }: { location: string }) => {
        console.log(`[시스템] ${location}의 날씨를 조회하는 함수가 실행되었습니다...`);
        return {
            location,
            temperature: "22도",
            condition: "맑음",
            humidity: "40%"
        };
    },
};

const genAI = new GoogleGenerativeAI(Deno.env.get("GOOGLE_API_KEY") || "");
const TOOLS = [
    {
        functionDeclarations: [
            {
                name: "get_current_weather",
                description: "특정 지역의 현재 날씨 및 기온 정보를 가져옵니다.",
                parameters: {
                    type: "object",
                    properties: {
                        location: {
                            type: "string",
                            description: "도시 이름 (예: 서울, 도쿄)",
                        },
                    },
                    required: ["location"],
                },
            },
        ],
    },
];


// 2. 모델 설정 및 도구(Tools) 등록
const model = genAI.getGenerativeModel({
    model: "gemini-3-flash-preview",
    tools: TOOLS,
});

const chat = model.startChat({
    history: [],
});

console.log("Gemini-3-flash-preview 함수 호출 답변 준비 완료!");
console.log("'서울 날씨 어때?'라고 물어보세요.");

In [ ]:
async function continuousChat() {
    console.log("--- 대화를 시작합니다. 'exit'를 입력하면 종료됩니다. ---");
    
    while (true) {
        const userInput = prompt("You:");
        
        if (userInput === null || userInput.trim().toLowerCase() === "exit") {
            console.log("대화를 종료합니다.");
            break;
        }

        if (userInput.trim() === "") continue;
        console.log("user :", userInput);
        
        try {
            let result = await chat.sendMessage(userInput);
            let response = await result.response;
            
            // 3. AI가 함수 호출을 요청했는지 확인
            const call = response.functionCalls()?.[0];
            
            if (call) {
                const apiResponse = functions[call.name](call.args);
                
                // 실행 결과를 다시 AI에게 보내서 최종 답변 생성
                result = await chat.sendMessage([{
                    functionResponse: {
                        name: call.name,
                        response: apiResponse
                    }
                }]);
                response = await result.response;
            }

            console.log("assistant :", response.text());
        } catch (error) {
            console.error("Error:", error);
            break;
        }
    }
}

await continuousChat();

In [ ]:
const history = await chat.getHistory();
console.log(history);